# The `ParsedConfig` Object

`ParsedConfig` is a python class used in `deode_prototype` to parse and validate config file options. The aim with this class is to provide an object that encapsulates configuration options and offers some (hopefully) handy methods to validate them and retrieve their values. This notebook will walk you through the setp and usage of obsects created with the `ParsedConfig` class.

# Getting Started

Start by importing `ParsedConfig`.

If you are writing code inside the `deode` project (i.e., at the level of the `deode/__init__.py` file ot below it), then you should use relative imports, such as

```python3
from .config_parser import ParsedConfig
```

If you are working outside the project structure, as in this document, then you should do a normal import, like:

In [ ]:
from deode.config_parser import ParsedConfig

# Creating `ParsedConfig` instances

## - From configs stored in a file (toml, yaml or json): The `.from_file` Method

In [ ]:
parsed_config = ParsedConfig.from_file("minimal_config_example.toml")

2023-02-17 12:42:10: Reading config file /home/a002216/src/git/2022_deode/deode_prototype/docs/minimal_config_example.toml


A couple of things happen when you execute this line of code:

1. The raw contents of the "minimal_config_example.toml" file are validated, in this case, against the default json schema (more about this later)
    - An exception will be raised if the validation fails.
2. Lists are converted into tuples
3. Dictionaries are recursively converted into `BaseConfig` objects (from which `ParsedConfig` inherits)
    - So that useful `ParsedConfig` instance methods, such as `.get_value`, work smoothly. More on such methods later.
4. If all goes well, then `main_config` is now a `ParsedConfig` object with entries validated according to the default json schema.

This is how your newly created object looks like:

In [ ]:
print(parsed_config)

ParsedConfig{
    "general": {
        "times": {
            "list": [
                "2020-01-01T12:00:00Z"
            ],
            "cycle_length": "PT3H"
        }
    }
}


The `ParsedConfig` object, as well as its `from_file` and `parse_obj` class methods, take on an optional argument `json_schema` that can be used to perform validation against arbitrary json schemas.

We are now going to parse the provided config file `minimal_config_example.toml`, using `ParsedConfig`'s `.from_file` method, validating it against the default JSON schema (defined in file `deode/data/config_file_schemas/main_config_schema.json`).








## - From configs stored in a `dict`-like object: The `.parse_obj` Method

In [ ]:
raw_config = {
    "general": {"times": {"list": ["2020-01-01T12:00:00Z"], "cycle_length": "PT3H"}}
}

parsed_config = ParsedConfig.parse_obj(raw_config)
print(parsed_config)

ParsedConfig{
    "general": {
        "times": {
            "list": [
                "2020-01-01T12:00:00Z"
            ],
            "cycle_length": "PT3H"
        }
    }
}


## - From configs stored in a `dict`-like object: Direct Instantiation

In [ ]:
raw_config = {
    "general": {"times": {"list": ["2020-01-01T12:00:00Z"], "cycle_length": "PT3H"}}
}

parsed_config = ParsedConfig(**raw_config)
print(parsed_config)

ParsedConfig{
    "general": {
        "times": {
            "list": [
                "2020-01-01T12:00:00Z"
            ],
            "cycle_length": "PT3H"
        }
    }
}


which, in fact, is equivalent to

In [ ]:
parsed_config = ParsedConfig(
    general={"times": {"list": ["2020-01-01T12:00:00Z"], "cycle_length": "PT3H"}}
)
print(parsed_config)

ParsedConfig{
    "general": {
        "times": {
            "list": [
                "2020-01-01T12:00:00Z"
            ],
            "cycle_length": "PT3H"
        }
    }
}


# Handy Methods/Attributes Available for `ParsedConfig` instances

There are a few convenience methods/attributes available for `ParsedConfig` instances, and more may be implemented in the future. For increased maintainability, let's generate a list of the name of these methods/attributes automatically :)

In [ ]:
import inspect

handy_methods = sorted(
    [
        name
        for name in dir(ParsedConfig(json_schema={}))
        if not name.startswith("_") and name not in list(parsed_config.items())
    ]
)
for method_name in handy_methods:
    print(method_name)

copy
dict
dumps
from_file
get_value
items
json_schema
parse_obj


More information about these methods is presented below -- but please take a look at the code and related unit tests if you are interested in knowing more about them.

In [ ]:
for method_name in handy_methods:
    method = getattr(parsed_config, method_name)
    print("============================")
    if inspect.ismethod(method):
        print(f"Signature: {method_name}{inspect.signature(method)}")
        print(f"Description: {inspect.getdoc(method)}")
    else:
        print(f"Name: {method_name}")
        print(f"Type: {type(method)}")
    print("")

Signature: copy(**kwargs)
Description: Return a copy of the instance. Same API as `copy` from class BasicConfig.

Signature: dict(descend_recursively=True)
Description: Return a dict representation of the instance and nested instances.

Signature: dumps(section='', style: Literal['toml', 'json', 'yaml'] = 'toml', include_metadata=False)
Description: Get a nicely printed version of the models. Excludes the metadata section.

Signature: from_file(config_path, json_schema=None)
Description: Read config file at location "config_path".

Args:
    config_path (typing.Union[pathlib.Path, str]): The path to the config file.
    json_schema (dict): JSON schema to be used for validation.

Returns:
    .config_parser.ParsedConfig: Parsed configs from config_path.

Signature: get_value(items)
Description: Recursively get the value of a config component.

This allows us to use self.get_value("foo.bar.baz") even if "bar" is, for
instance, a dictionary or any obj that implements a "getitem" method.



# Inspecting the Used JSON Schema

Just access the `.json_schema` attribute of the created instance:

In [ ]:
print(parsed_config.json_schema)

{
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "MainConfigSchema",
    "description": "Model for validation of the data in the main config file.",
    "type": "object",
    "additionalProperties": true,
    "properties": {
        "general": {
            "$ref": "#/definitions/GeneralSectionModel"
        }
    },
    "required": [
        "general"
    ],
    "definitions": {
        "GeneralSectionModel": {
            "title": "GeneralSectionModel",
            "description": "Model for the 'general' section.",
            "type": "object",
            "properties": {
                "times": {
                    "title": "times",
                    "description": "Model for the 'general.times' section.",
                    "type": "object",
                    "properties": {
                        "start": {
                            "title": "Start",
                            "type": "string",
                            "format": "date-time"
  

The json schema is also included in the instance's `repr`, which may be useful on some occasions:

In [ ]:
print(repr(parsed_config))

ParsedConfig(**{
    "general": {
        "times": {
            "list": [
                "2020-01-01T12:00:00Z"
            ],
            "cycle_length": "PT3H"
        }
    }
}, json_schema={
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "MainConfigSchema",
    "description": "Model for validation of the data in the main config file.",
    "type": "object",
    "additionalProperties": true,
    "properties": {
        "general": {
            "$ref": "#/definitions/GeneralSectionModel"
        }
    },
    "required": [
        "general"
    ],
    "definitions": {
        "GeneralSectionModel": {
            "title": "GeneralSectionModel",
            "description": "Model for the 'general' section.",
            "type": "object",
            "properties": {
                "times": {
                    "title": "times",
                    "description": "Model for the 'general.times' section.",
                    "type": "object",
                    

# Config Validation Against Arbitrary JSON Schemas

Say you wish to perform validation of configs agains a schema other than the default one. As an example, we'll use a sample json schema obtained from the official [JSON Schema webpage](https://json-schema.org/learn/miscellaneous-examples.html#describing-geographical-coordinates) on 2022-02-16:

In [ ]:
my_custom_schema = {
    "$id": "https://example.com/geographical-location.schema.json",
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Longitude and Latitude Values",
    "description": "A geographical coordinate.",
    "required": ["latitude", "longitude"],
    "type": "object",
    "properties": {
        "latitude": {"type": "number", "minimum": -90, "maximum": 90},
        "longitude": {"type": "number", "minimum": -180, "maximum": 180},
    },
}

Now let's define our raw config data and parse/validate it against our custom JSON schema using `ParsedConfig`.

For convenience, we'll use [the `.parse_obj` Method](#Creating-ParsedConfig-instances-from-configs-stored-in-a-dict-like-object:-The-.parse_obj-Method), but the same logic applies for the other instantiation methods.

In [ ]:
raw_config = {"latitude": 0, "longitude": 0}
parsed_config = ParsedConfig.parse_obj(raw_config, json_schema=my_custom_schema)
print(parsed_config)

ParsedConfig{
    "latitude": 0,
    "longitude": 0
}


And that's it :)

# Example of Validation Error

Let's pass an invalid value to the "latitude" field. A `ConfigFileValidationError` should be raised, as this value is not within the allowed range according to the used json schema. The error message should indicate the nature and location of the error.

In [ ]:
from deode.config_parser import ConfigFileValidationError

raw_config = {"latitude": 450000, "longitude": 0}
try:
    parsed_config = ParsedConfig.parse_obj(raw_config, json_schema=my_custom_schema)
except ConfigFileValidationError as err:
    print(err)

"latitude" must be smaller than or equal to 90. Received type "int" with value "450000".


# Default Values

Default values are populated upon config parsing. To demonstrate this, let's alter slightly our example json schema by:
1. Adding a default value to the `latitude` field
2. Removing the field from the list of required properties

In [ ]:
my_custom_schema = {
    "$id": "https://example.com/geographical-location.schema.json",
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Longitude and Latitude Values",
    "description": "A geographical coordinate.",
    "required": ["longitude"],
    "type": "object",
    "properties": {
        "latitude": {"type": "number", "minimum": -90, "maximum": 90, "default": 450000},
        "longitude": {"type": "number", "minimum": -180, "maximum": 180, "default": 0},
    },
}

In [ ]:
parsed_config = ParsedConfig.parse_obj({"longitude": 120}, json_schema=my_custom_schema)
print(parsed_config)

ParsedConfig{
    "longitude": 120,
    "latitude": 450000
}


Note, from this example that **default values are not validated**. It is up to whoever designs the json schema to ensure that the default values are reasonable.

# Extra Fields (Fields That Are Present in Raw Data but Not Pre-Configured in the JSON Schema)

Anything that is not validated against a json schema or against a pre-defined field in a json schema (e.g., extra fields in the main config file that are not defined in data/config_file_schemas/main_config_schema.json) will be passed as is. No validation is performed on these:

In [ ]:
raw_config = {
    "latitude": 0,
    "longitude": 0,
    "unforeseen_field": "This is passed 'as is'.",
}
parsed_config = ParsedConfig.parse_obj(raw_config, json_schema=my_custom_schema)
print(parsed_config)

ParsedConfig{
    "latitude": 0,
    "longitude": 0,
    "unforeseen_field": "This is passed 'as is'."
}


# Bypassing Validation

You can bypass validation entirely by passing an empty json schema to the `ParsedConfig` constructors. 

In [ ]:
my_custom_schema = {}
raw_config = {"foo": "bar", "baz": ["qux"], "quux": {"corge": 42}}
parsed_config = ParsedConfig.parse_obj(raw_config, json_schema=my_custom_schema)
print(parsed_config)

ParsedConfig{
    "foo": "bar",
    "baz": [
        "qux"
    ],
    "quux": {
        "corge": 42
    }
}


This can be useful, for example, in order to leverage the convenience methods defined for instances of the `ParsedConfig` class.

# Further Resources

Further information and examples, templates, etc. can be found at the following locations: 
- [Links to various JSON schemas](https://github.com/DEODE-NWP/Deode-Prototype/pull/60#issuecomment-1426985437) used for validation of config_exp.h in Harmonie
- [Links to helper scripts](https://github.com/DEODE-NWP/Deode-Prototype/pull/60#issuecomment-1426989737) to minimise the need to write all the schema files by hand
- The code's own [unit tests for the `config_parser.py` file](https://github.com/DEODE-NWP/Deode-Prototype/blob/develop/tests/unit/test_config_parser.py)